In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e10/sample_submission.csv
/kaggle/input/playground-series-s3e10/train.csv
/kaggle/input/playground-series-s3e10/test.csv


# Imports

In [2]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Get data

In [3]:
train_data = pd.read_csv('/kaggle/input/playground-series-s3e10/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s3e10/test.csv')

X = train_data.copy()
X.drop(['id', 'Class'], axis=1, inplace=True)

y = train_data['Class']

no_id_test_data = test_data.drop('id', axis=1)

display(X)
display(y)

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve
0,133.171875,59.716081,0.043133,-0.703383,54.917224,70.084438,0.749798,-0.649512
1,87.093750,36.257973,0.435469,2.266057,3.417224,21.865069,7.039330,52.686251
2,112.640625,39.818393,0.379639,0.922306,2.730769,15.689690,8.193471,85.649785
3,120.679688,45.918448,-0.098490,0.011775,2.696488,20.954662,8.183874,70.332899
4,134.070312,57.720107,-0.107772,-0.573335,1.107860,11.255051,16.107748,308.753765
...,...,...,...,...,...,...,...,...
117559,132.843750,56.748838,-0.060070,-0.554084,4.054348,27.844144,6.564423,44.442664
117560,112.578125,52.539271,0.179580,-0.306961,1.637960,15.331913,11.384718,142.535470
117561,119.757812,49.980013,0.064402,-0.270822,3.877926,19.788559,6.959740,56.367789
117562,105.789062,46.986595,0.441426,0.372466,2.097826,17.170612,9.442445,99.074539


0         0
1         0
2         0
3         0
4         0
         ..
117559    0
117560    0
117561    0
117562    0
117563    0
Name: Class, Length: 117564, dtype: int64

# Split data

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

# Missing data

In [5]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

print(f'There are {len(cols_with_missing)} columns with missing values')

for i in cols_with_missing:
    print(i)

There are 0 columns with missing values


# Categorical data

In [6]:
cat_cols = [col for col in X_train.columns if X_train[col].dtype=='object']

print(f'There are {len(cat_cols)} columns with categorical values')

for i in cat_cols:
    print(i)

There are 0 columns with categorical values


# Score

In [7]:
def get_score(n_estimators_=100, learning_rate_=0.5, max_depth_=6):
    model = XGBRegressor(n_estimators=n_estimators_, max_depth=max_depth_, learning_rate=learning_rate_, n_jobs=4)
    model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)
    predictions = model.predict(X_valid)
    return mean_absolute_error(predictions, y_valid)

# Find optimal n_estimators

In [8]:
display(X_train)
display(y_train)

best_score=100000000
best_n_estimators=100
best_learning_rate=0.1
best_max_depth=6

for n_estimators in range(100, 401, 100):
    score = get_score(n_estimators_=n_estimators)
    print(f'n_estimators {n_estimators}: {score}')
    if score<best_score:
        best_score=score
        best_n_estimators=n_estimators
        
for learning_rate in [0.005, 0.01, 0.025, 0.05, 0.1]:
    score = get_score(n_estimators_=best_n_estimators, learning_rate_=learning_rate)
    print(f'learning rate {learning_rate}: {score}')
    if score<best_score:
        best_score=score
        best_learning_rate=learning_rate
        
for max_depth in [2, 4, 6, 8, 10]:
    score = get_score(n_estimators_=best_n_estimators, learning_rate_=learning_rate, max_depth_=max_depth)
    print(f'max depth {max_depth}: {score}')
    if score<best_score:
        best_score=score
        best_max_depth=max_depth

,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve
103604,105.875000,53.308946,0.269539,-0.386832,2.076087,16.998601,10.817910,134.398109
110816,129.593750,46.172514,0.191217,0.159850,1.869565,12.818230,10.924381,149.232406
226,105.617188,38.692503,0.402597,1.534129,1.892140,14.763218,10.747616,135.928673
93684,117.296875,50.913404,0.202720,-0.334631,1.498328,13.443961,12.386947,181.786775
18349,54.664062,34.735224,3.835513,14.983223,20.372074,53.290736,2.673749,6.341108
...,...,...,...,...,...,...,...,...
28905,146.039062,46.175514,-0.353368,0.007735,5.127926,25.931197,5.955098,38.187569
21198,112.968750,52.076098,0.107616,-0.394914,125.673913,78.548414,-0.936519,-1.392165
71524,132.710938,53.002211,-0.087713,-0.334637,3.383779,20.760330,7.920423,72.826626
93662,122.945312,53.538865,-0.078139,-0.517064,2.971572,17.159629,8.829858,91.154473


103604    0
110816    0
226       0
93684     0
18349     1
         ..
28905     0
21198     0
71524     0
93662     0
77916     0
Name: Class, Length: 88173, dtype: int64

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


n_estimators 100: 0.015857645385722912


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


...

max depth 6: 0.015542691125151525


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


max depth 8: 0.019513094752332392


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


max depth 10: 0.017448733728336274


# Submission

In [9]:
# model = XGBRegressor(n_estimators = best_n_estimators, learning_rate=0.05, n_jobs=4)
model = XGBRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth, learning_rate=best_learning_rate)
model.fit(X, y)
predictions = model.predict(no_id_test_data)

submission = pd.DataFrame({'id': test_data.id, 'Class': predictions})

display(submission)
display(submission.head())

submission.to_csv('submission.csv', index=False)

print('Successful')

,id,Class
0,117564,0.000350
1,117565,0.000696
2,117566,0.000238
3,117567,0.047605
4,117568,0.002022
...,...,...
78372,195936,0.000345
78373,195937,0.000345
78374,195938,0.000345
78375,195939,0.049031


,id,Class
0,117564,0.000350
1,117565,0.000696
2,117566,0.000238
3,117567,0.047605
4,117568,0.002022


Successful
